Install Ngrok, SQLAlchemy, Flask-Session.

Mount Google Drive for File Access & setup token.

In [ ]:
!pip install pyngrok
!pip install flask_sqlalchemy
!pip install Flask-Session

import os, getpass

from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define where your token lives (or would live)
TOKEN_PATH = "/content/drive/MyDrive/ngrok_token.txt"

# 3. Load or prompt
if os.path.exists(TOKEN_PATH):
    with open(TOKEN_PATH, "r") as f:
        ngrok_token = f.read().strip()
    print("Loaded Ngrok auth token from file.")
else:
    ngrok_token = getpass.getpass("Enter your Ngrok auth token: ")

# 4. Export into env so flask_ngrok can pick it up
os.environ["NGROK_AUTH_TOKEN"] = ngrok_token


Latest Server Code

In [ ]:
# Core Python modules
import os
import sys
import random
import logging
import sqlite3
from secrets import token_hex

# Flask and extensions
from flask import Flask, request, jsonify, render_template, session
from flask_session import Session

# Third-party libraries
from pyngrok import ngrok
from werkzeug.security import generate_password_hash, check_password_hash

logging.basicConfig(level=logging.DEBUG)

# Define project paths
BASE       = "/content/drive/MyDrive/colab-web-app"
TEMPLATES  = os.path.join(BASE, "templates")
STATIC     = os.path.join(BASE, "static")
ROUTES_DIR = os.path.join(BASE, "routes")
DB_PATH    = os.path.join(BASE, "database", "site.db")

# Confirm route folder exists
print("ROUTES folder →", ROUTES_DIR)
!ls "{ROUTES_DIR}"

# Ensure Python can import from there
sys.path.insert(0, ROUTES_DIR)

# Create Flask app & register Blueprint
app = Flask(
    __name__,
    template_folder=TEMPLATES,
    static_folder=STATIC
)

# Inject your DB_PATH into app.config **before** registering routes
app.config['DB_PATH'] = DB_PATH

from main_routes import main_bp
app.register_blueprint(main_bp)

from auth_routes import auth_bp
app.register_blueprint(auth_bp)

from admin_routes import admin_bp
app.register_blueprint(admin_bp)

# Session & Secret Key
app.config["SECRET_KEY"]   =  token_hex(32)
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# Database Connection Function
def get_db_connection():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

from functools import wraps
from flask import session, redirect, url_for, flash
from flask import jsonify


# Ngrok Authentication & Tunnel Setup
auth_token = os.getenv("NGROK_AUTH_TOKEN")
if auth_token:
    ngrok.set_auth_token(auth_token)

public_url = ngrok.connect(8000)
print(f'Access the web app here: {public_url}')

# Run Flask Server
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)


Misc. SQL.

Show All Users.

In [ ]:
import sqlite3
from tabulate import tabulate

with sqlite3.connect("/content/drive/MyDrive/colab-web-app/database/site.db") as conn:
  cur = conn.cursor()

cur.execute("SELECT id, username, email, is_admin FROM users;")
rows = cur.fetchall()

print(tabulate(rows, headers=["ID", "Username", "Email", "Is Admin"], tablefmt="github"))

|   ID | Username   | Email                  |   Is Admin |
|------|------------|------------------------|------------|
|    1 | alice      | alice@example.com      |          1 |
|    2 | bob        | bob@example.com        |          0 |
|    3 | carol      | carol@example.com      |          0 |
|    4 | dave       | dave@example.com       |          0 |
|    5 | eve        | eve@example.com        |          1 |
|    6 | frank      | frank@example.com      |          0 |
|    7 | gina       | gina@example.com       |          1 |
|    8 | hannah     | hannah@example.com     |          0 |
|    9 | ian        | ian@example.com        |          0 |
|   10 | jess       | jess@example.com       |          0 |
|   11 | test_admin | test_admin@example.com |          1 |


Populate 10.

In [ ]:
import sqlite3
from werkzeug.security import generate_password_hash

DB_PATH = "/content/drive/MyDrive/colab-web-app/database/site.db"

# Sample user records: (id, username, email, password_hash, is_admin)
users = [
    (1,  "alice",  "alice@example.com",  generate_password_hash("alicepass"),  1),
    (2,  "bob",    "bob@example.com",    generate_password_hash("bobpass"),    0),
    (3,  "carol",  "carol@example.com",  generate_password_hash("carolpass"),  0),
    (4,  "dave",   "dave@example.com",   generate_password_hash("davepass"),   0),
    (5,  "eve",    "eve@example.com",    generate_password_hash("evepass"),    1),
    (6,  "frank",  "frank@example.com",  generate_password_hash("frankpass"),  0),
    (7,  "gina",   "gina@example.com",   generate_password_hash("ginapass"),   1),
    (8,  "hannah", "hannah@example.com", generate_password_hash("hannahpass"), 0),
    (9,  "ian",    "ian@example.com",    generate_password_hash("ianpass"),    0),
    (10, "jess",   "jess@example.com",   generate_password_hash("jesspass"),   0),
]

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.executemany(
        "INSERT INTO users (id, username, email, password, is_admin) VALUES (?, ?, ?, ?, ?);",
        users
    )
    conn.commit()

print("Inserted 10 sample users into the users table.")


Inserted 10 sample users into the users table.


In [ ]:
import sqlite3
from werkzeug.security import generate_password_hash

DB_PATH = "/content/drive/MyDrive/colab-web-app/database/site.db"

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute(
        """
        INSERT INTO users (id, username, email, password, is_admin)
        VALUES (?, ?, ?, ?, ?);
        """,
        (
            11,
            "test_admin",
            "test_admin@example.com",
            generate_password_hash("test_admin"),
            1
        )
    )
    conn.commit()

print("Inserted test_admin successfully.")


Inserted test_admin successfully.


Print Schema.

In [ ]:
import sqlite3
from tabulate import tabulate

DB_PATH = "/content/drive/MyDrive/colab-web-app/database/site.db"

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    # Grab the table structure
    cur.execute("PRAGMA table_info(users);")
    schema = cur.fetchall()

# PRAGMA table_info returns rows like:
# (cid, name, type, notnull, dflt_value, pk)
print(tabulate(
    schema,
    headers=["CID","Column","Type","NotNull","Default","PK"],
    tablefmt="github"
))

|   CID | Column   | Type    |   NotNull |   Default |   PK |
|-------|----------|---------|-----------|-----------|------|
|     0 | id       | INTEGER |         0 |           |    1 |
|     1 | username | TEXT    |         1 |           |    0 |
|     2 | email    | TEXT    |         1 |           |    0 |
|     3 | password | TEXT    |         1 |           |    0 |
|     4 | is_admin | INTEGER |         1 |         0 |    0 |


In [ ]:
import sqlite3

DB_PATH = "/content/drive/MyDrive/colab-web-app/database/site.db"

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute(
        "DELETE FROM users WHERE username = ?;",
        ("test_admin",)
    )
    conn.commit()

print("Removed test_admin successfully.")

Removed test_admin successfully.


Routes Moved to Files.

In [ ]:
!find /content/drive -maxdepth 3 -type d -name routes

!ls "/content/drive/My Drive/colab-web-app/routes"

import os
print(os.getcwd())



/content/drive/MyDrive/colab-web-app/routes
auth_routes.py	main_routes.py	__pycache__
/content
